In [1]:
import gym
import numpy as np
import pandas as pd
import sys
from math import copysign
from enum import Enum
from copy import copy
import warnings
from typing import Optional, Dict, Callable, List, Union
import os
import glob

from src.envs import BaseTradingEnv
from src.utils import DataLoader, Preprocessor, backtest
from src.envs.reward_func import equity_log_return_reward

Loading BokehJS ...

In [2]:
if len(glob.glob(f"./data/^N225/*.csv")) != 0:
    data = DataLoader.load_data(f"./data/^N225_ohlcv.csv")
    features = DataLoader.load_data(f"./data/^N225_features.csv")
else:
    os.makedirs(f"./data/^N225", exist_ok=True)
    data = DataLoader.fetch_data("^N225", interval="1d")
    data, features = Preprocessor.extract_features(data)
    data.to_csv(f"./data/^N225/ohlcv.csv")
    features.to_csv(f"./data/^N225/features.csv")

data = data.iloc[-3000:]
features = features.iloc[-3000:]

data_train, features_train, data_eval, features_eval = Preprocessor.train_test_split(
    data,
    features,
    test_size=0.5,
    scaling=True,
)

In [3]:
class Actions(Enum):
    Sell = 0
    Buy = 1
    # Neutral = 2
    def perform(env: "BaseTradingEnv", action):
        if action == Actions.Sell.value:
            if env.position.is_long:
                env.position.close()
            elif env.position.is_short:
                pass
            else:
                env.sell(size=env.trade_size, sl=env.sl_price)

        elif action == Actions.Buy.value:
            if env.position.is_short:
                env.position.close()
            elif env.position.is_long:
                pass
            else:
                env.buy(size=env.trade_size, sl=env.sl_price)
        else:
            raise ValueError

In [4]:
env = BaseTradingEnv(data_train, features_train, actions=Actions)

In [5]:
env.action_space.sample()

0

In [6]:
backtest(env)

Start                     2009-07-31 00:00:00
End                       2015-09-14 00:00:00
Duration                   2236 days 00:00:00
Exposure Time [%]                        99.6
Equity Final [$]                 97058.696047
Equity Peak [$]                 110698.640117
Return [%]                          -2.941304
Buy & Hold Return [%]               73.467162
Return (Ann.) [%]                   -0.500296
Volatility (Ann.) [%]                9.683625
Sharpe Ratio                        -0.003263
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -25.798395
Avg. Drawdown [%]                   -3.194848
Max. Drawdown Duration     1880 days 00:00:00
Avg. Drawdown Duration      169 days 00:00:00
# Trades                                  507
Win Rate [%]                        48.323471
Best Trade [%]                       5.438528
Worst Trade [%]                    -13.129653
Avg. Trade [%]                    

In [7]:
env.render()

=============== Environment (2015-09-11 00:00:00) ===============
Price: 18264.220703125
Assets: 96917.10229687642
Equity: 96917.10229687642
Orders: []
Trades: []
Position: <Position: 0 (0 trades)>
Closed Trades: [<Trade size=4 time=5-6 price=10422.501933593749-10524.259765625 pnl=407>, <Trade size=4 time=7-8 price=10596.045420898436-10435.0 pnl=-644>, <Trade size=-4 time=9-10 price=10506.673239257812-10597.330078125 pnl=-363>, <Trade size=4 time=11-12 price=10278.87896191406-10284.9599609375 pnl=24>, <Trade size=-4 time=13-17 price=10193.796-10497.3603515625 pnl=-1214>, <Trade size=-4 time=18-19 price=10629.070250976563-10473.9697265625 pnl=620>, <Trade size=4 time=20-21 price=10544.673788085936-10492.5302734375 pnl=-209>, <Trade size=-4 time=22-23 price=10270.179500976563-10214.6396484375 pnl=222>, <Trade size=-4 time=24-25 price=10176.923241210938-10320.9404296875 pnl=-576>, <Trade size=4 time=26-27 price=10403.62369921875-10312.1396484375 pnl=-366>, <Trade size=-4 time=28-29 price=